# Module 3.2 Add-on: `circuit.parameters` shows only unassigned symbols  

In [25]:

# Metadata
import sys, platform
from importlib.metadata import version, PackageNotFoundError
def safe_ver(pkg):
    try:
        return version(pkg)
    except PackageNotFoundError:
        return "not installed"
print("Python:", sys.version.split()[0])
print("Qiskit:", safe_ver("qiskit"))
print("qiskit-ibm-runtime:", safe_ver("qiskit-ibm-runtime"))


Python: 3.12.9
Qiskit: 2.2.0
qiskit-ibm-runtime: 0.41.1


In [26]:

# Imports (Qiskit 2.x APIs)
from math import pi
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2, EstimatorV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

backend = FakeSherbrooke()



## Conceptual Overview  

`Parameter` is a scalar symbol; `ParameterVector` is an ordered collection of  
symbols. `assign_parameters` binds symbolic parameters to numeric values (or to  
other symbols). It accepts dictionaries (including vector-as-key), or a sequence  
aligned to the circuit's parameter order. The `inplace` flag chooses whether to  
modify the circuit or return a new one.  

Always compile the bound circuit for the target backend (here a fake backend)  
before running `SamplerV2`/`EstimatorV2`.  


### 3.2.0  Create and bind a single `Parameter`  

**Docs:**  
- Parameter:  
https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.Parameter  

Notice parameters may be used more than once, and not all parameters need to be used

In [27]:
from qiskit.circuit import Parameter
from math import pi
phi = Parameter("φ")
g = Parameter("g")
qc = QuantumCircuit(1)
qc.rx(phi, 0)
qc.ry(phi/2, 0)
qc.measure_all()
qc.draw()

┌───────┐┌─────────┐ ░ ┌─┐
     q: ┤ Rx(φ) ├┤ Ry(φ/2) ├─░─┤M├
        └───────┘└─────────┘ ░ └╥┘
meas: 1/════════════════════════╩═
                                0


### 3.2.1  Build structured families with `ParameterVector`  

**Docs:**  
- ParameterVector:  
  https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.ParameterVector
- ParameterVector can be used to assign vectors of values as parameters.  It's a more convenient way to manage many parameters at once.

In [29]:

vec = ParameterVector("θ", 2)a
qc = QuantumCircuit(1)
qc.ry(vec[0], 0)
qc.rz(vec[1], 0)
qc.measure_all()

assign_map = {vec[0]: 0.2, vec[1]: 0.1}
bqc = qc.assign_parameters(assign_map)
tqc = transpile(bqc, backend)
sampler = SamplerV2(mode=backend)
job = sampler.run([tqc], shots=512)
results = job.result()
print(results[0].data.meas.get_counts())


{'0': 507, '1': 5}



## 3.2.3 Assigning Parameters

- We create a circuit using only a subset of symbols from a `ParameterVector`.  
`circuit.parameters` lists the **unassigned** symbols still present. After  
`assign_parameters(..., inplace=True)`, the set becomes empty if all used  
symbols were bound.
- We will also show that you cannot assign parameters that are not used in the circuit.
- There are three ways to assign parameters via `assign_parameters` method:
1)  `assign_parameters` with a dictionary (scalar keys)
2)  `assign_parameters` using a `ParameterVector` as the key
3)  `assign_parameters` with a positional sequence  (same order as they were created)


### 3.2.3.1  `assign_parameters` with a dictionary (scalar keys)  

Map each `Parameter` to a numeric value. This is explicit and common.  

In [31]:

θ, ϕ = Parameter("θ"), Parameter("φ")
qc = QuantumCircuit(1)
qc.rx(θ, 0)
qc.rz(ϕ, 0)
qc.measure_all()

mapped = qc.assign_parameters({θ: 0.3, ϕ: 0.15})
print("params now:", sorted(p.name for p in mapped.parameters))



params now: []



### 3.2.3.2  `assign_parameters` using a `ParameterVector` as the key  

In [32]:

thetas = ParameterVector("θ", 3)
qc = QuantumCircuit(1)
qc.rx(thetas[0], 0)
qc.ry(thetas[1], 0)
qc.rz(thetas[2], 0)
qc.measure_all()

mapped = qc.assign_parameters({thetas: [0.1, 0.2, 0.3]})
print("Remaining unbound params:", sorted(p.name for p in mapped.parameters))

Remaining unbound params: []



### 3.2.3.3  `assign_parameters` with a positional sequence  

In [34]:

a, b = Parameter("a"), Parameter("b")
qc = QuantumCircuit(1)
qc.ry(a, 0)
qc.rz(b, 0)
qc.measure_all()

print("Parameter order:", [p.name for p in qc.parameters])
mapped = qc.assign_parameters([0.25, 0.5])  # a -> 0.25, b -> 0.5
print("After assign:", sorted(p.name for p in mapped.parameters))

Parameter order: ['a', 'b']
After assign: []


## 3.2.4  `inplace` argument: return-new vs modify-in-place  

`inplace` keyword may be used to return a copy of the circuit  
if set to `False`.  If set to `True`, then nothing is returned   
and the circuit is overwritten (modified in place).

In [6]:

theta = ParameterVector('theta', 3)  # theta[0], theta[1], theta[2]
phi = ParameterVector('phi', 2)      # not used here

cir = QuantumCircuit(2, 2)
cir.h(0)
cir.rx(theta[0], 0)   # uses theta[0]
cir.ry(theta[1], 1)   # uses theta[1]

print("Before assignment, cir.parameters should show theta[0] and theta[1]:")
print(sorted(p.name for p in cir.parameters))

# ---- Uncomment to show error ----
# Try to assign all entries of 'theta' using vector-as-key mapping 
# cir.assign_parameters({theta: [0.1, 0.1, 0.1]}, inplace=True)  

cir.assign_parameters({theta[0]: 0.1, theta[1]:0.1}, inplace=True)  

print("\nAfter in-place assignment, cir.parameters should be empty:")
print(sorted(p.name for p in cir.parameters))


Before assignment, cir.parameters should show theta[0] and theta[1]:
['theta[0]', 'theta[1]']

After in-place assignment, cir.parameters should be empty:
[]


Notice that even though we did not use `theta[2]` it doesn't appear when we print   
`cir.parameters` because only parameters that are used in the circuit are printed.

### **Common pitfalls**

- The `ParameterVector` constructor takes as input in this order:
  `ParameterVector(<name>, <size>)`.
- This is in contrast to `QuantumRegister` which takes as input:
  `QuantumRegister(<size>, <name)`.



## 3.2.5 Multiple Choice Questions  

**Q1.** Which call creates `θ[0]`, `θ[1]`, `θ[2]`?  
A. `[Parameter('θ')]*3`  
B. `ParameterVector('θ', 3)`  
C. `Parameter('θ', 3)`  
D. `ParameterVector(3, 'θ')`  

**Q2.** What does `inplace=True` do for `assign_parameters`?  
A. Returns a new circuit.  
B. Modifies the circuit and typically returns `None`.  
C. Throws unless all parameters are bound.  
D. Has no effect.  

**Q3.** Which mapping binds all entries of a vector at once?  
A. `{θ: [0.1, 0.2, 0.3]}` where `θ = ParameterVector('θ', 3)`  
B. `{θ[0]: [0.1, 0.2, 0.3]}`  
C. `{θ: 0.1}`  
D. `[0.1, 0.2, 0.3, 0.4]`  

<details>  
<summary><b>Answer Key</b></summary>  

Q1: B  
Q2: B  
Q3: A  

</details>  